In [1]:
!pip install mysql-connector-python

In [2]:
import mysql.connector

try:
    conn1 = mysql.connector.connect(
        host="localhost", # MySQL 서버1 주소
        user="root",
        password="1234"
    )
    conn2 = mysql.connector.connect(
        host="210.119.14.56", # MySQL 서버2 주소
        user="root",
        password="1234"
    )    
    if conn1.is_connected():
        print("MySQL 서버1에 성공적으로 연결되었습니다.")
    if conn2.is_connected():
        print("MySQL 서버2에 성공적으로 연결되었습니다.")
except mysql.connector.Error as err:
    print("MySQL 연결 실패:", err)
finally:
    if all([conn1.is_connected(),conn2.is_connected()]):
        print("연결중...")

MySQL 서버1에 성공적으로 연결되었습니다.
MySQL 서버2에 성공적으로 연결되었습니다.
연결중...


In [3]:
# 커서 생성
cursor1 = conn1.cursor()
cursor2 = conn2.cursor()
que = "CREATE DATABASE IF NOT EXISTS smdb0759"
cursor1.execute(que)
cursor2.execute(que)
conn1.close()
conn2.close()

In [4]:
# MySQL 에 연결
conn1 = mysql.connector.connect(
        host="localhost", # MySQL 서버 주소
        user="root",
        password="1234",
        database="smdb0759" # 사용할 데이터 베이스
    )
conn2 = mysql.connector.connect(
        host="210.119.14.56", # MySQL 서버 주소
        user="root",
        password="1234",
        database="smdb0759" # 사용할 데이터 베이스
    )
cursor1 = conn1.cursor()
cursor2 = conn2.cursor()
print("MySQL 연결 성공!")

MySQL 연결 성공!


In [5]:
# 테이블 생성
query = "CREATE TABLE IF NOT EXISTS users(id INT AUTO_INCREMENT PRIMARY KEY ,name VARCHAR(50), address VARCHAR(100), type VARCHAR(50), latitbude VARCHAR(50), slong VARCHAR(50))"
cursor1.execute(query)
cursor2.execute(query)
print("테이블 생성완료")

테이블 생성완료


In [6]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from datetime import datetime
from bs4 import BeautifulSoup as bs
import pandas as pd
import time

In [7]:
# Chrome 옵션 설정
options = Options()
Options.binary_location = "./chrome-win64/chrome-win64/chrome.exe" # chrome 경로 설정
options.add_argument("--no-sandbox") # 샌드박스 설정
options.add_argument("--no-disable-dev-shm-usage") # 공유 메모리 설정
options.add_argument("--window-size=1920,1080") #화면크기설정, 반응형 웹 대비
#options.add_argument("--no-disable-gpu") #gpu 비활성화

#chrome 드라이버 설정
service = Service("chromedriver.exe")
driver = webdriver.Chrome(service=service, options=options)
driver.get("https://www.starbucks.co.kr/")
print(driver.title)

# 요소찾기
hover_element = driver.find_element(By.CSS_SELECTOR, "#gnb > div > nav > div > ul > li.gnb_nav03 > h2 > a")
menu_option = driver.find_element(By.CSS_SELECTOR, "#gnb > div > nav > div > ul > li.gnb_nav03 > div > div > div > ul:nth-child(1) > li.gnb_sub_ttl > a")

# 마우스 호버 엑션
actions = ActionChains(driver)
actions.move_to_element(hover_element).perform()

# 약간의 시간을 기다린 후 클릭
time.sleep(1)
menu_option.click()
time.sleep(5)
print("매장검색중")

# 입력창 선택 후 검색
driver.find_element(By.CSS_SELECTOR,"#container > div > form > fieldset > div > section > article.find_store_cont > article > header.loca_search > h3 > a").click() # '지역검색' 을 선택
print("지역검색중")
time.sleep(1)
driver.find_element(By.CSS_SELECTOR,"#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(6) > a").click() # '부산'을 클릭
print("부산검색중")
time.sleep(1)
driver.find_element(By.CSS_SELECTOR,"#mCSB_2_container > ul > li:nth-child(1) > a").click() # '부산'을 클릭
print("부산전체로드중")
time.sleep(1)
html = driver.page_source
print(html)
driver.quit()  # 브라우저 닫기

Starbucks Korea
매장검색중
지역검색중
부산검색중
부산전체로드중
<html lang="ko"><head>
		








<meta http-equiv="X-UA-Compatible" content="IE=edge">
<meta charset="utf-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no">
<meta property="og:type" content="website">
<meta property="og:title" content="Starbucks">
<meta property="og:url" content="https://www.starbucks.co.kr/">
<meta property="og:image" content="https://image.istarbucks.co.kr/common/img/kakaotalk.png">
<meta property="og:description" content="Starbucks">

<title id="titleJoin">Starbucks Korea</title><script src="https://connect.facebook.net/ko_KR/sdk.js?hash=0e587aa3e6381b3e0320064d5b68ae97" async="" crossorigin="anonymous"></script><script id="facebook-jssdk" src="//connect.facebook.net/ko_KR/sdk.js"></script><script type="text/javascript" async="" src="https://www.googletagmanager.com/gtag/js?id=G-WC8Q3C59QP&amp;cx=c&amp;_slc=1"></script><script async="" src="//ww

In [8]:
# 현재 페이지 정보 가져오기
star_list=[]
soup = bs(html, 'html.parser')
# 본문내용 가져오기
contents = soup.select("#mCSB_3_container > ul > li")

for c in contents:
    result = ( c['data-name'], c.select_one('p.result_details').text , c.select_one('i')['class'][0], c['data-lat'],c['data-long'] )
    star_list.append(result)    


    # 여러개의 데이터 삽입
    sql = "INSERT INTO users (name, address, type, latitbude, slong) VALUES (%s, %s, %s, %s, %s)"
    m_values=[( c['data-name'], c.select_one('p.result_details').text , c.select_one('i')['class'][0], c['data-lat'],c['data-long'] )]
    cursor1.executemany(sql, m_values)
    cursor2.executemany(sql, m_values)
    conn1.commit()
    conn2.commit()
print(star_list)
print('여러개의 데이터 삽입 완료, ID: ')

[('금정남산동DT', '부산광역시 금정구 중앙대로 1989 (남산동)1522-3232', 'pin_generalDT', '35.2621786292157', '129.091325043351'), ('금정구청DT', '부산광역시 금정구 중앙대로 1730 (부곡동)1522-3232', 'pin_generalDT', '35.2388135211636', '129.092985849499'), ('온천장역DT', '부산광역시 금정구 식물원로 13 (장전동)1522-3232', 'pin_generalDT', '35.224006', '129.08656'), ('부산대정문', '부산광역시 금정구 부산대학로64번길 12 (장전동)1522-3232', 'pin_general', '35.23212165114605', '129.0844959292725'), ('부산대역', '부산광역시 금정구 장전로12번길 64, 1~4층 (장전동)1522-3232', 'pin_general', '35.23052180284867', '129.08869433543111'), ('금정이마트', '부산광역시 금정구 중앙대로1841번길 24 (구서동)1522-3232', 'pin_general', '35.249819818853915', '129.09078227309334'), ('금정구서', '부산광역시 금정구 금강로 441 (구서동)1522-3232', 'pin_general', '35.246908383445785', '129.08873844189125'), ('부산서동', '부산광역시 금정구 서동로 173 (서동)1522-3232', 'pin_general', '35.21463924851766', '129.10558442984572'), ('용호동W스퀘어', '부산광역시 남구 분포로 145 (용호동) 더블유스퀘어동 1019호,1020호,1021호,2011호 2012호1522-3232', 'pin_general', '35.1337011', '129.113324'), ('동명대DT', '부산광역시 남구 신선